# train

In [2]:
import numpy as np
import pandas as pd
import json
from time import time
pd.set_option('display.max_columns', None)

In [3]:
training_raw = pd.read_csv('smzdm_jingxuan.csv', encoding='utf-8', index_col=0)
training_raw.head()

,_id,action,article_button_name,article_category,article_channel,article_channel_class,article_channel_id,article_channel_url,article_collection,article_comment,article_date,article_date_full,article_id,article_is_anonymous,article_is_sold_out,article_is_timeout,article_link,article_link_arr,article_link_name,article_link_nofollow,article_mall,article_mall_domain,article_mall_link,article_price,article_referrals,article_tags,article_taobao_url,article_tese_tags,article_timesort,article_title,article_title_mode,article_unworthy,article_url,article_user_url,article_worthy,article_yh_type,create_date,gtm,is_black_five,mall_more_info,price
0,7521534,buy,去购买,{'category_url': 'http://www.smzdm.com/fenlei/...,国内,z-tag-guonei,3,http://www.smzdm.com/youhui/,45,82,07-11 22:56,2017-07-11 22:56:00.000,7521534,2,NaN,NaN,https://detail.tmall.com/item.htm?id=538505214759,[],NaN,"rel=""nofollow""",天猫精选,tmall.com,http://www.smzdm.com/mall/tmall/,199元包邮（需用券）,商家自荐,"[{'os_num': 0, 'jy_num': 0, 'url_nicktitle': '...",{'href': 'https://go.smzdm.com/57cd4860f0eb72b...,"[{'os_num': 0, 'jy_num': 0, 'url_nicktitle': '...",1499784964,magic eyes 萝莉动漫名器 处女新娘,0,24,http://www.smzdm.com/p/7521534/,NaN,220,youhui,2017-08-11 17:46:53.756,"{'object': ""{'name':'magic eyes 萝莉动漫名器 处女新娘','...",0,[],199.00
1,7515444,dislike,去购买,{'category_url': 'http://www.smzdm.com/fenlei/...,海淘,z-tag-haitao,3,http://haitao.smzdm.com/,9,1,07-11 15:32,2017-07-11 15:32:00.000,7515444,2,NaN,NaN,https://go.smzdm.com/f1942e10b2bbd903/ca_aa_ht...,[],NaN,"rel=""nofollow""",HQhair,hqhair.com,http://www.smzdm.com/mall/hqhair/,买3免1+满40英镑包直邮,商家自荐,"[{'os_num': 0, 'jy_num': 0, 'url_nicktitle': '...",{'href': 'https://go.smzdm.com/f1942e10b2bbd90...,"[{'os_num': 0, 'jy_num': 0, 'url_nicktitle': '...",1499758324,海淘活动：HQhair 精选 FIRST AID BEAUTY 护肤专场,0,1,http://www.smzdm.com/p/7515444/,NaN,4,huodong,2017-08-11 17:46:55.809,"{'object': ""{'name':'海淘活动：HQhair 精选 FIRST AID ...",0,"[{'info': '', 'title': '英国', 'icon': 'http://r...",3.00
2,7512991,dislike,去购买,{'category_url': 'http://www.smzdm.com/fenlei/...,跨境,z-tag-kuajing,3,http://www.smzdm.com/kuajing/,2,2,07-11 15:31,2017-07-11 15:31:00.000,7512991,2,NaN,NaN,https://go.smzdm.com/a44af7f577c0bab2/ca_aa_yh...,[],NaN,"rel=""nofollow""",亚马逊海外购,amazon.cn,http://www.smzdm.com/mall/amazon_b/,￥566.47+￥169.94含税直邮（约￥736.41）,商家自荐,"[{'os_num': 0, 'jy_num': 0, 'url_nicktitle': '...",{'href': 'https://go.smzdm.com/a44af7f577c0bab...,"[{'os_num': 0, 'jy_num': 0, 'url_nicktitle': '...",1499758278,Graco 葛莱 Glider Elite 婴儿摇篮,0,1,http://www.smzdm.com/p/7512991/,NaN,4,youhui,2017-08-11 17:46:55.809,"{'object': ""{'name':'Graco 葛莱 Glider Elite 婴儿摇...",0,[],566.47
3,7520760,dislike,去购买,{'category_url': 'http://www.smzdm.com/fenlei/...,跨境,z-tag-kuajing,3,http://www.smzdm.com/kuajing/,7,10,07-11 15:39,2017-07-11 15:39:00.000,7520760,0,NaN,NaN,https://go.smzdm.com/b8fc991e8aded30e/ca_aa_yh...,[],NaN,"rel=""nofollow""",亚马逊海外购,amazon.cn,http://www.smzdm.com/mall/amazon_b/,￥280.06+ ￥33.34税费（约￥313.4）,日百母婴小小值,"[{'os_num': 0, 'jy_num': 0, 'url_nicktitle': '...",{'href': 'https://go.smzdm.com/b8fc991e8aded30...,"[{'os_num': 0, 'jy_num': 0, 'url_nicktitle': '...",1499758762,中亚Prime会员：THERMOS 膳魔师 FHI-250DS 儿童保温吸管杯 250ml *2件,0,9,http://www.smzdm.com/p/7520760/,http://zhiyou.smzdm.com/member/5421866207/,2,youhui,2017-08-11 17:46:55.974,"{'object': ""{'name':'中亚Prime会员：THERMOS 膳魔师 FHI...",0,[],280.06
4,7516050,buy,去购买,{'category_url': 'http://www.smzdm.com/fenlei/...,跨境,z-tag-kuajing,3,http://www.smzdm.com/kuajing/,240,170,07-10 21:29,2017-07-10 21:29:00.000,7516050,0,NaN,NaN,https://go.smzdm.com/e030543a04505c46/ca_aa_yh...,[],NaN,"rel=""nofollow""",亚马逊海外购,amazon.cn,http://www.smzdm.com/mall/amazon_b/,￥370.19+￥44.05含税直邮（合￥414.24）,zero_tan,"[{'os_num': 0, 'jy_num': 0, 'url_nicktitle': '...",{'href': 'https://go.smzdm.com/e030543a04505c4...,"[{'os_num': 0, 'jy_num': 0, 'url_nicktitle': '...",1499693390,中亚Prime会员：PHILIPS 飞利浦 HX8911/04 声波电动牙刷,0,10,http://www.smzdm.com/p/7516050/,http://

In [4]:
def extract_tags(tag_str):
    tags = json.loads(tag_str.replace("'", "\"").replace(": None,",": \"None\","))
    tags_name = [tag['name'] for tag in tags]
    return ', '.join(tags_name)

training_raw[['article_tags']].applymap(extract_tags).head()

,article_tags
0,成人向
1,"直邮, 支付宝"
2,全球PrimeDay母婴日百
3,"中亚Prime会员, 拼单"
4,"绝对值, 爱美丽"


In [5]:
def preprocess_data(raw_data):
    raw_data.replace(['?'], [None], inplace=True)
#     raw_data.dropna(how='all', axis=1, inplace=True)
    raw_data.drop(labels=['_id', 'article_id', 'article_url', 'article_price', 'article_link',
                          'article_date', 'article_channel_class', 'mall_more_info', 'article_button_name',
                          'article_link_name', 'article_category', 'gtm', 'article_tese_tags',
                          'article_link_arr', 'article_link_nofollow', 'article_taobao_url',
                          'article_user_url'],
                  axis=1, inplace=True)
    
    raw_data['article_worthy'].replace({None: 0}, inplace=True)
    raw_data = raw_data.fillna(0)
    
    raw_data[['article_tags']] = raw_data[['article_tags']].applymap(extract_tags)
    
    raw_data[['article_worthy', 'article_unworthy', 'article_is_timeout']] = \
        raw_data[['article_worthy', 'article_unworthy', 'article_is_timeout']].apply(pd.to_numeric, errors='coerce')
    raw_data[['article_date_full', 'create_date']] = \
        raw_data[['article_date_full', 'create_date']].apply(pd.to_datetime, errors='ignore').astype('int64')
        
    from sklearn.preprocessing import LabelEncoder
    encoder = LabelEncoder()
    raw_data['article_mall'] = encoder.fit_transform(raw_data['article_mall'])
    raw_data['article_mall_domain'] = encoder.fit_transform(raw_data['article_mall_domain'])
    raw_data['article_mall_link'] = encoder.fit_transform(raw_data['article_mall_link'].astype('str'))
    raw_data['article_channel'] = encoder.fit_transform(raw_data['article_channel'])
    raw_data['article_referrals'] = encoder.fit_transform(raw_data['article_referrals'].astype('str'))
    raw_data['article_yh_type'] = encoder.fit_transform(raw_data['article_yh_type'].astype('str'))
    raw_data['article_channel_url'] = encoder.fit_transform(raw_data['article_channel_url'])
    
    from sklearn.preprocessing import StandardScaler
    raw_data['article_date_full'] = StandardScaler().fit_transform(raw_data['article_date_full'].values.reshape(-1, 1))
    raw_data['create_date'] = StandardScaler().fit_transform(raw_data['create_date'].values.reshape(-1, 1))
    raw_data['article_timesort'] = StandardScaler().fit_transform(raw_data['article_timesort'].values.reshape(-1, 1))
    
    return raw_data

In [6]:
def process_text(raw_data, vect_title=None, vect_tag=None):
    from sklearn.feature_extraction.text import CountVectorizer
    if vect_title is None:
        vect_title = CountVectorizer(min_df=2)
        vect_title.fit(raw_data['article_title'])
    if vect_tag is None:
        vect_tag = CountVectorizer()
        vect_tag.fit(raw_data['article_tags'])
    title_feature_matrix = vect_title.transform(raw_data['article_title']).toarray()
    tag_feature_matrix = vect_tag.transform(raw_data['article_tags']).toarray()
    feature_matrix = raw_data.drop(['article_title', 'article_tags'], axis=1)
    sample_data = np.hstack((feature_matrix, title_feature_matrix, tag_feature_matrix))
    return sample_data, vect_title, vect_tag

In [7]:
df = pd.read_csv('smzdm_jingxuan.csv', encoding='utf-8', index_col=0)
df_sample = df.drop(['action'], axis=1)
df_target = df['action']

In [8]:
df_sample = preprocess_data(df_sample)
sample_data, vect_title, vect_tag = process_text(df_sample)
sample_data.shape

/Users/dd/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


(445, 569)

In [9]:
sample_target = df_target.replace(to_replace=dict(buy=2, view=1, dislike=0))
sample_target.shape

(445,)

In [10]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [25]:
clf = LogisticRegression()
print('cv score: ', cross_val_score(clf, sample_data, sample_target, cv=10).mean())

cv score:  0.68398158271


In [12]:
clf = DecisionTreeClassifier(min_samples_split=15)
print('cv score: ', cross_val_score(clf, sample_data, sample_target, cv=10).mean())

cv score:  0.581946430073


In [13]:
clf = SVC(kernel='rbf')
print('cv score: ', cross_val_score(clf, sample_data, sample_target, cv=10).mean())

cv score:  0.633836515011


In [14]:
clf = GaussianNB()
print('cv score: ', cross_val_score(clf, sample_data, sample_target, cv=10).mean())

cv score:  0.432757968211


In [15]:
clf = AdaBoostClassifier(n_estimators=15, learning_rate=0.5)
print('cv score: ', cross_val_score(clf, sample_data, sample_target, cv=10).mean())

cv score:  0.681563787739


In [16]:
clf = GradientBoostingClassifier(n_estimators=15, learning_rate=0.5)
print('cv score: ', cross_val_score(clf, sample_data, sample_target, cv=10).mean())

cv score:  0.623105710201


In [17]:
from sklearn.model_selection import RandomizedSearchCV
search_param = {'n_estimators': np.arange(1, 50, 1), 'learning_rate': np.arange(0.1, 10, 0.1)}
t_start = time()
clf = RandomizedSearchCV(AdaBoostClassifier(), search_param, cv=10, scoring='accuracy', n_iter=50, random_state=1, n_jobs=-1)
clf.fit(sample_data, sample_target)
print('search time: ', (time() - t_start))
print(clf.best_score_)
print(clf.best_estimator_)

search time:  16.165643215179443
0.710112359551
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.70000000000000007, n_estimators=6,
          random_state=None)


In [117]:
t_start = time()
search_param = {'n_estimators': range(1, 50, 1),
                'max_depth': range(1, 25, 1), 'min_samples_split': range(5, 50, 1),
#                 'min_samples_split': range(10, 50, 1), 'min_samples_leaf': range(10, 50, 1),
#                 'max_features': range(5, 50, 1),
#                 'learning_rate': np.arange(0.1, 10, 0.1)
                }
clf = RandomizedSearchCV(GradientBoostingClassifier(), search_param, cv=10, scoring='accuracy', n_iter=50, random_state=1, n_jobs=-1)
clf.fit(sample_data, sample_target)
print('search time: ', (time() - t_start))
print(clf.best_score_)
print(clf.best_estimator_)

search time:  183.4003040790558
0.707865168539
GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=23,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=17, min_weight_fraction_leaf=0.0,
              n_estimators=3, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)


In [50]:
X_train, X_test, y_train, y_test = train_test_split(sample_data, sample_target, test_size=0.2, random_state=2017)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(356, 569) (356,) (89, 569) (89,)


In [110]:
class Ensemble(object):
    def __init__(self, n_folds, stacker, base_models):
        self.n_folds = n_folds
        self.stacker = stacker
        self.base_models = base_models
    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)
        folds = KFold(n_splits=self.n_folds, shuffle=True, random_state=2016)
        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):
            S_test_i = np.zeros((T.shape[0], self.n_folds))
            for j, (train_idx, test_idx) in enumerate(folds.split(X)):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
                # y_holdout = y[test_idx]
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_holdout)[:]
                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict(T)[:]
            S_test[:, i] = S_test_i.mean(1)
        self.stacker.fit(S_train, y)
        y_pred = self.stacker.predict(S_test)[:]
        return y_pred

In [150]:
es = Ensemble(5, LogisticRegression(), [LogisticRegression(), DecisionTreeClassifier(min_samples_split=15), SVC(kernel='rbf')])
es_pred = es.fit_predict(X_train, y_train, X_test)
print('auc: {}'.format(1.0 * sum(es_pred == y_test) / y_test.shape[0]))

auc: 0.797752808988764


In [127]:
from sklearn.ensemble import RandomForestClassifier
es = Ensemble(5, LogisticRegression(), [RandomForestClassifier(max_depth=20), DecisionTreeClassifier(min_samples_split=15), SVC(kernel='rbf')])
es_pred = es.fit_predict(X_train, y_train, X_test)
print('auc: {}'.format(1.0 * sum(es_pred == y_test) / y_test.shape[0]))

auc: 0.8089887640449438


# predict

In [ ]:
df_predict = pd.read_csv('predict.csv', encoding='utf-8', index_col=0)
df_show = df_predict

In [ ]:
df_predict = preprocess_data(df_predict)
predict_data, _, _ = process_text(df_predict, vect_title, vect_tag)
predict_data.shape

In [ ]:
prediction = clf.predict(predict_data)
prediction

In [ ]:
df_show = df_show.drop(labels=['_id', 'article_id', 'article_url', 'article_price', 'article_link',
                          'article_date', 'article_channel_class', 'mall_more_info', 'article_button_name',
                          'article_link_name', 'article_category', 'gtm', 'article_tese_tags',
                          'article_link_arr', 'article_link_nofollow', 'article_taobao_url',
                          'article_user_url'], axis=1)
df_show = df_show.fillna(0)
df_show[['article_tags']] = df_show[['article_tags']].applymap(extract_tags)
df_show['predict_action'] = prediction
df_show[['predict_action']] = df_show[['predict_action']].replace(to_replace={2:'buy', 1:'view', 0:'dislike'})

In [ ]:
df_show